In [169]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [170]:
with open('data/curated/player_data.json', 'r') as f:
    player_data = json.load(f)

In [171]:
df = pd.DataFrame(player_data)

In [172]:
df = df.astype({
    'match_id': 'string',
    'account_id': 'string',
    'hero_id': 'string',
    'radiant_team_id': 'string',
    'dire_team_id': 'string',
    'win': 'boolean',
    'firstblood_claimed': 'boolean',
    'team_id': 'string',
    'enemy_team_id': 'string',
    'valveId': 'string',
})

In [173]:
q1_glicko = df['glicko2_rating'].quantile(0.25)
df['glicko2_rating'] = df['glicko2_rating'].fillna(q1_glicko)
q1_enemy_glicko = df['enemy_glicko2_rating'].quantile(0.25)
df['enemy_glicko2_rating'] = df['enemy_glicko2_rating'].fillna(q1_enemy_glicko)

In [174]:
mask = df['rating_advantage'].isna()
df.loc[mask, 'rating_advantage'] = df.loc[mask, 'glicko2_rating'] - df.loc[mask, 'enemy_glicko2_rating']

In [175]:
df = df.sort_values(['account_id', 'match_start_time'])


In [176]:
SEQ_LEN=10

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15530 entries, 1126 to 14947
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  string 
 1   match_start_time         15530 non-null  int64  
 2   account_id               15530 non-null  string 
 3   name                     15530 non-null  object 
 4   fantasy_role             15530 non-null  object 
 5   hero_id                  15530 non-null  string 
 6   hero_variant             15530 non-null  object 
 7   isRadiant                15530 non-null  bool   
 8   radiant_team_id          15530 non-null  string 
 9   dire_team_id             15530 non-null  string 
 10  tournament_start_date    15530 non-null  object 
 11  duration                 15530 non-null  int64  
 12  win                      15530 non-null  boolean
 13  kills                    15530 non-null  int64  
 14  deaths                  

In [178]:
not_features = [
    'match_id', 'account_id', 'name', 'radiant_team_id', 'dire_team_id',
    'tournament_start_date', 'teamName', 'valveId', 'enemy_teamName', 'enemy_valveId', 'fantasy_score', 'team_id'
]

In [179]:
df['isRadiant'] = df['isRadiant'].astype(int)
df['win'] = df['win'].astype(int)
df['firstblood_claimed'] = df['firstblood_claimed'].astype(int)

In [180]:
df = pd.get_dummies(df, columns=['fantasy_role', 'hero_id', 'hero_variant'])

In [181]:
#features = [col for col in df.columns if col not in not_features]
features = [col for col in df.columns if col not in not_features and pd.api.types.is_numeric_dtype(df[col])]#df[features].isna().sum()
df[features] = df[features].fillna(df[features].mean())

In [182]:
X_seqs = []
y_seqs = []

for _, player_df in df.groupby('account_id'):
    player_df = player_df.reset_index(drop=True)
    for i in range(len(player_df) - SEQ_LEN):
        X_seqs.append(player_df.loc[i:i+SEQ_LEN-1, features].values)
        y_seqs.append(player_df.loc[i+SEQ_LEN, 'fantasy_score'])

X_seqs = np.array(X_seqs)
y_seqs = np.array(y_seqs)

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X_seqs, y_seqs, test_size=0.1)#, random_state=42)

In [184]:
model = models.Sequential([
    layers.Input(shape=(SEQ_LEN, X_train.shape[2])),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 64)             │       127,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,809 (499.25 KB)

 Trainable params: 127,809 (499.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 198.6640 - val_loss: 117.5329
Epoch 2/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99.3632 - val_loss: 57.6405
Epoch 3/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 50.0023 - val_loss: 33.3157
Epoch 4/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 30.7009 - val_loss: 25.7798
Epoch 5/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 24.7010 - val_loss: 24.0478
Epoch 6/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 23.8063 - val_loss: 23.7639
Epoch 7/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 23.5272 - val_loss: 23.7346
Epoch 8/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 24.0976 - val_loss: 23.7344
Epoch 9/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 23.4380 - val_loss: 23.7264
Epoch 10/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 24.0381 - val_loss: 23.7305
Epoch 11/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 23.6372 - val_loss: 23.7239
Epoch 12/20
340/340 ━━━━━━━━

In [185]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
print("LSTM RMSE:", root_mean_squared_error(y_test, y_pred))

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
LSTM RMSE: 4.872984309544811


In [186]:
def make_symmetric_matchups(matchups):
    symmetric = []
    for t1, t2 in matchups:
        symmetric.append((t1, t2))
        symmetric.append((t2, t1))
    return symmetric

In [187]:
def predict_players_for_matchups_lstm(df, model, matchup_list, features, seq_len=10, lineup_dir="data/lineups"):
    results = []
    for team_id, enemy_team_id in matchup_list:
        lineup_path = os.path.join(lineup_dir, f"team_lineup_{team_id}.json")
        try:
            with open(lineup_path, "r", encoding="utf-8") as f:
                lineup = [p for p in json.load(f) if p.get("is_current_team_member")]
        except FileNotFoundError:
            continue

        for player in lineup:
            account_id = str(player["account_id"])
            player_hist = df[df["account_id"] == account_id].sort_values("match_start_time")
            if len(player_hist) < seq_len:
                continue

            seq_df = player_hist.iloc[-seq_len:].copy()
            seq_df["team_id"] = team_id
            seq_df["enemy_team_id"] = enemy_team_id

            # Preprocessing (same as training)
            seq_df['isRadiant'] = seq_df['isRadiant'].astype(int)
            seq_df['win'] = seq_df['win'].astype(int)
            seq_df['firstblood_claimed'] = seq_df['firstblood_claimed'].astype(int)
            for col in features:
                if col not in seq_df.columns:
                    seq_df[col] = 0
            seq_df = seq_df[features]
            seq_df = seq_df.fillna(seq_df.mean()).astype(np.float32)
            print(seq_df.head())
            print(seq_df.sum())
            X_pred = seq_df.values.reshape(1, seq_len, len(features))
            pred_score = model.predict(X_pred)[0][0]
            results.append({
                "account_id": account_id,
                "name": player.get("name", ""),
                "team_id": team_id,
                "enemy_team_id": enemy_team_id,
                "predicted_fantasy_score": pred_score
            })
    return pd.DataFrame(results)

In [188]:
print(X_test[0])

[[1.72969812e+09 1.00000000e+00 2.17200000e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.72976604e+09 0.00000000e+00 3.37700000e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.72977144e+09 1.00000000e+00 3.12000000e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.73005674e+09 0.00000000e+00 3.05500000e+03 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [1.73006160e+09 1.00000000e+00 2.15600000e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.73012191e+09 1.00000000e+00 2.13400000e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [189]:
matchups = [('7119388', '8261500'), ('8597976', '7119388'), ('8597976', '8261500')] # Spirit vs. Xtreme Gaming, Spirit vs. Talon
symmetric_matchups = make_symmetric_matchups(matchups)
result = predict_players_for_matchups_lstm(df, model, symmetric_matchups, features, seq_len=SEQ_LEN)

       match_start_time  isRadiant  duration  win  kills  deaths  last_hits  \
15341      1.752055e+09        1.0    2254.0  1.0    3.0     2.0       67.0   
15206      1.752138e+09        0.0    1577.0  1.0    6.0     2.0       22.0   
15181      1.752142e+09        1.0    1928.0  1.0    1.0     3.0       39.0   
14821      1.752658e+09        1.0    3897.0  1.0    3.0     6.0      206.0   
14816      1.752664e+09        0.0    2339.0  1.0    4.0     3.0       76.0   

       denies  gold_per_min  towers_killed  ...  hero_variant_95, 2  \
15341     5.0         358.0            0.0  ...                 0.0   
15206     3.0         339.0            0.0  ...                 0.0   
15181     2.0         346.0            1.0  ...                 0.0   
14821     0.0         446.0            1.0  ...                 0.0   
14816     2.0         378.0            1.0  ...                 0.0   

       hero_variant_96, 1  hero_variant_96, 2  hero_variant_97, 2  \
15341                 0.0    

In [190]:
result

,account_id,name,team_id,enemy_team_id,predicted_fantasy_score
0,113331514,Miposhka,7119388,8261500,14.626884
1,321580662,Yatoro,7119388,8261500,14.626884
2,302214028,Collapse,7119388,8261500,14.626884
3,106305042,Larl,7119388,8261500,14.626884
4,847565596,rue,7119388,8261500,14.626884
5,203351055,Malik,7119388,8261500,14.626884
6,137129583,Xm,8261500,7119388,14.626884
7,157475523,XinQ,8261500,7119388,14.626884
8,129958758,Xxs,8261500,7119388,14.626884
9,898754153,Ame,8261500,7119388,14.626884


In [191]:
#print(np.std(X_test, axis=0)) 

In [192]:
print(np.std(y_train))

4.904062458292332
